# How to handle multiple queries

:::info Prerequisites

This guide assumes familiarity with the following:

- [Query analysis](/docs/tutorials/query_analysis)

:::

Sometimes, a query analysis technique may allow for multiple queries to be generated. In these cases, we need to remember to run all queries and then to combine the results. We will show a simple example (using mock data) of how to do that.

## Setup

### Install dependencies

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/openai zod chromadb
</Npm2Yarn>
```

### Set environment variables

```
OPENAI_API_KEY=your-api-key

# Optional, use LangSmith for best-in-class observability
LANGSMITH_API_KEY=your-api-key
LANGCHAIN_TRACING_V2=true
```

### Create Index

We will create a vectorstore over fake information.

In [ ]:
import { Chroma } from "@langchain/community/vectorstores/chroma"
import { OpenAIEmbeddings } from "@langchain/openai"
import "chromadb";

const texts = ["Harrison worked at Kensho", "Ankush worked at Facebook"]
const embeddings = new OpenAIEmbeddings({ model: "text-embedding-3-small" })
const vectorstore = await Chroma.fromTexts(
    texts,
    {},
    embeddings,
    {
        collectionName: "multi_query"
    }
)
const retriever = vectorstore.asRetriever(1);

## Query analysis

We will use function calling to structure the output. We will let it return multiple queries.

In [3]:
import { z } from "zod";

const searchSchema = z.object({
    queries: z.array(z.string()).describe("Distinct queries to search for")
}).describe("Search over a database of job records.");

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [4]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableSequence, RunnablePassthrough } from "@langchain/core/runnables";

const system = `You have the ability to issue search queries to get information to help answer user information.

If you need to look up two distinct pieces of information, you are allowed to do that!`;

const prompt = ChatPromptTemplate.fromMessages([
    ["system", system],
    ["human", "{question}"],
])
const llmWithTools = llm.withStructuredOutput(searchSchema, {
  name: "Search"
});
const queryAnalyzer = RunnableSequence.from([
  {
      question: new RunnablePassthrough(),
  },
  prompt,
  llmWithTools
]);

We can see that this allows for creating multiple queries

In [5]:
await queryAnalyzer.invoke("where did Harrison Work")

{ queries: [ "Harrison" ] }

In [6]:
await queryAnalyzer.invoke("where did Harrison and ankush Work")

{ queries: [ "Harrison work", "Ankush work" ] }

## Retrieval with query analysis

So how would we include this in a chain? One thing that will make this a lot easier is if we call our retriever asyncronously - this will let us loop over the queries and not get blocked on the response time.

In [7]:
import { RunnableConfig, RunnableLambda } from "@langchain/core/runnables";

const chain = async (question: string, config?: RunnableConfig) => {
    const response = await queryAnalyzer.invoke(question, config);
    const docs = [];
    for (const query of response.queries) {
        const newDocs = await retriever.invoke(query, config);
        docs.push(...newDocs);
    }
    // You probably want to think about reranking or deduplicating documents here
    // But that is a separate topic
    return docs;
}

const customChain = new RunnableLambda({ func: chain });

In [8]:
await customChain.invoke("where did Harrison Work")

[ Document { pageContent: "Harrison worked at Kensho", metadata: {} } ]

In [10]:
await customChain.invoke("where did Harrison and ankush Work")

[
  Document { pageContent: "Harrison worked at Kensho", metadata: {} },
  Document { pageContent: "Ankush worked at Facebook", metadata: {} }
]

## Next steps

You've now learned some techniques for handling multiple queries in a query analysis system.

Next, check out some of the other query analysis guides in this section, like [how to deal with cases where no query is generated](/docs/how_to/query_no_queries).